# M&M benchmark XI

This benchmark is an improvments over the [previous one](https://gaow.github.io/mvarbvs/analysis/20191108_MNM_Benchmark.html), in the following espects.

1. Use both small $R=5$ and large $R=45$ simulations to compare if merely increasing number of conditions messes it up.
2. Simulate even simpler: 1 effect and using 1 grid for effect covariance such that the prior is no longer a mixture.
3. Analyzing it with L = 1 or L = 2
4. Assess CS overlap at both variable and CS level
5. Add oracle residual covariance method
6. Turn on ELBO computation and add a score to check for convergence; although for now we know that missing data ELBO computation can be problematic.

**Note**: due to 5 above, results with missing data does not make sense anyways because ELBO-based check for convergence is not yet working with missing data. I am removing results for missing data computation from displaying for now.

## Conclusion

1. Under simple situation, that is, small number of effects, small enough L, and no missing data, increasing $R=5$ to $R=45$ does not seem to harm.
    - And as expected from the simulated senario, increased number of conditions help with the power due to sharing of effects by magnitude.
2. CS overlapping situation exists, and gets worse as $R$ increases, even for `L=2` and lower power setting compared to previous simulation.
    - Using an oracle covariance does not seem to have helped.
3. FLASH based covariance hurts the power.

## Next steps for this investigation

1. Add to this benchmark $L=10$ run ... because effect number has changed it is difficult to compare with previous results using $L=10$.
2. Missing data situation: when ELBO computation is not involved and when residual covariance is simply diag, missing data handling should be really easy -- not sure how to check for that "bug".
3. Should we fix ELBO computation for when there is missing data? If so, how?
4. Not shown on this benchmark: ELBO has some bugs with using EE model (alpha = 0) -- some results are NaN. I suspect it has something to do with when both `bhat` and `sbhat` are NA in univariate regression. Currently we set `bhat=0` and `sbhat=1E6`, for those situations.

The corresponding DSC code are from `35d4456379236bd48c0eef79142168661cfbd83b` and to be reproduced as follows:

```
./finemap.dsc --host dsc_mnm.yml -o mnm_20191114
```

In [1]:
%cd ~/GIT/mvarbvs/dsc_mnm

/project2/mstephens/gaow/mvarbvs/dsc_mnm

In [8]:
start_time <- Sys.time()
out = dscrutils::dscquery('mnm_20191114', targets = c('simulate.n_traits', 'mnm.resid_method', 'mnm.missing_Y', 'mnm.alpha', 'mnm.L', 'susie_scores.total', 'susie_scores.valid', 'susie_scores.size', 'susie_scores.purity', 'susie_scores.top', 'susie_scores.n_causal', 'susie_scores.included_causal', 'susie_scores.overlap_var', 'susie_scores.overlap_cs','susie_scores.false_pos_cond_discoveries', 'susie_scores.false_neg_cond_discoveries', 'susie_scores.true_cond_discoveries', 'susie_scores.converged'), verbose = F)
end_time <- Sys.time()

In [9]:
end_time - start_time

Time difference of 16.87277 secs

In [10]:
head(out)

DSC,simulate.n_traits,mnm.resid_method,mnm.missing_Y,mnm.alpha,mnm.L,susie_scores.total,susie_scores.valid,susie_scores.size,susie_scores.purity,susie_scores.top,susie_scores.n_causal,susie_scores.included_causal,susie_scores.overlap_var,susie_scores.overlap_cs,susie_scores.false_pos_cond_discoveries,susie_scores.false_neg_cond_discoveries,susie_scores.true_cond_discoveries,susie_scores.converged
1,5,oracle,TRUE,1,2,0,0,0,0.0000000,0,1,0,0,0,0,0,0,TRUE
1,5,oracle,TRUE,1,2,0,0,0,0.0000000,0,1,0,0,0,0,0,0,TRUE
1,5,oracle,TRUE,1,2,0,0,0,0.0000000,0,1,0,0,0,0,0,0,TRUE
1,5,oracle,TRUE,1,2,0,0,0,0.0000000,0,1,0,0,0,0,0,0,TRUE
1,5,oracle,TRUE,1,2,0,0,0,0.0000000,0,1,0,0,0,0,0,0,TRUE
1,5,oracle,TRUE,1,2,1,1,3,0.9816157,1,1,1,0,0,0,0,5,TRUE


In [11]:
dim(out)

[1] 3048   19

In [12]:
saveRDS(out, '../data/finemap_output.20191114.rds')

In [15]:
res = out[,-1]
colnames(res) = c('n_traits', 'resid_method', 'missing', 'EZ_model', 'L', 'total', 'valid', 'size', 'purity', 'top_hit', 'total_true', 'total_true_included', 'overlap_var', 'overlap_cs', 'false_positive_cross_cond', 'false_negative_cross_cond', 'true_positive_cross_cond', 'elbo_converged')

### Purity of CS

Purity is higher for $R=45$ simply due to higher power; because in this simulation there is no FDR issue.

In [41]:
purity = aggregate(purity~n_traits + resid_method + missing + L, res, mean)
purity = purity[which(purity$missing==FALSE),-3]
purity = purity[order(purity$n_traits),]
purity

,n_traits,resid_method,L,purity
1,5,diag,1,0.6564498
3,5,flash,1,0.3902171
5,5,oracle,1,0.6976840
13,5,diag,2,0.5844912
15,5,flash,2,0.3103291
17,5,oracle,2,0.6452938
2,45,diag,1,0.9960895
4,45,flash,1,0.5181123
6,45,oracle,1,0.9964939
14,45,diag,2,0.9926904


### Power of CS

Focusing on $L = 2$ to evaluate overlapping CS status. In this case there still exists overlaps between CS, but not as many as with $L=10$. Overlapping status got worse when increased $R$.

In [45]:
total_true_included = aggregate(total_true_included ~ n_traits + resid_method + missing + L, res, sum)
total_true = aggregate(total_true ~  n_traits + resid_method + missing + L, res, sum)
cs_overlap = aggregate(overlap_cs ~  n_traits + resid_method + missing + L, res, mean)
snp_overlap = aggregate(overlap_var ~  n_traits + resid_method + missing + L, res, mean)
power = merge(total_true_included, total_true, by = c( 'n_traits' , 'resid_method' , 'missing' , 'L'))
power = merge(power, cs_overlap,  by = c( 'n_traits' , 'resid_method' , 'missing' , 'L'))
power = merge(power, snp_overlap,  by = c( 'n_traits' , 'resid_method' , 'missing' , 'L'))
power$power = round(power$total_true_included/power$total_true,3)
power$overlap_cs = round(power$overlap_cs, 3)
power$overlap_var = round(power$overlap_var, 3)
power = power[which(power$missing==FALSE),-3]
power = power[order(power$n_traits),]
power = power[order(power$L),]
power

,n_traits,resid_method,L,total_true_included,total_true,overlap_cs,overlap_var,power
13,5,diag,1,92,127,0.000,0.000,0.724
17,5,flash,1,58,127,0.000,0.000,0.457
21,5,oracle,1,97,127,0.000,0.000,0.764
1,45,diag,1,127,127,0.000,0.000,1.000
5,45,flash,1,71,127,0.000,0.000,0.559
9,45,oracle,1,127,127,0.000,0.000,1.000
14,5,diag,2,85,127,0.008,0.433,0.669
18,5,flash,2,46,127,0.000,0.000,0.362
22,5,oracle,2,92,127,0.008,0.110,0.724
2,45,diag,2,127,127,0.024,0.331,1.000


### FDR of CS

In [46]:
valid = aggregate(valid ~ n_traits + resid_method + missing + L, res, sum)
total = aggregate(total ~ n_traits + resid_method + missing + L, res, sum)
fdr = merge(valid, total, by = c( 'n_traits' , 'resid_method' , 'missing' , 'L'))
fdr$fdr = round((fdr$total - fdr$valid)/fdr$total,3)
fdr = fdr[which(fdr$missing==FALSE),-3]
fdr = fdr[order(fdr$n_traits),]
fdr

,n_traits,resid_method,L,valid,total,fdr
13,5,diag,1,92,92,0.000
14,5,diag,2,86,86,0.000
17,5,flash,1,58,58,0.000
18,5,flash,2,46,46,0.000
21,5,oracle,1,97,97,0.000
22,5,oracle,2,93,93,0.000
1,45,diag,1,127,127,0.000
2,45,diag,2,130,130,0.000
5,45,flash,1,71,72,0.014
6,45,flash,2,71,72,0.014


## Convergence

Based on ELBO. In principle all runs should converge by ELBO. If it is not converged, then it means ELBO is not non-increasing.

It is only relevant to focus on $L>1$. For without missing data the runs do converge wrt ELBO.

In [47]:
elbo_converged = aggregate(elbo_converged~n_traits + resid_method + missing + L, res, mean)
#elbo_converged = elbo_converged[which(elbo_converged$missing==FALSE),-3]
elbo_converged = elbo_converged[which(elbo_converged$L!=1),-4]
elbo_converged = elbo_converged[order(elbo_converged$n_traits),]
elbo_converged

,n_traits,resid_method,missing,elbo_converged
13,5,diag,FALSE,1.00000000
15,5,flash,FALSE,1.00000000
17,5,oracle,FALSE,1.00000000
19,5,diag,TRUE,0.96062992
21,5,flash,TRUE,0.96062992
23,5,oracle,TRUE,0.96850394
14,45,diag,FALSE,1.00000000
16,45,flash,FALSE,1.00000000
18,45,oracle,FALSE,1.00000000
20,45,diag,TRUE,0.08661417
